In [66]:
import pandas as pd
import os
from matplotlib import pyplot as plt

In [67]:
fin = "/h/dsmith/physionet_data/preppedf/physionet_data.parquet"
df = pd.read_parquet(fin, engine="pyarrow")
npts = 10
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,SepsisEver,id
0,60.0,98.0,36.200,135.0,109.0,72.0,17.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,1.0,0.0,0,27338
1,60.0,98.0,36.200,135.0,109.0,72.0,17.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,2.0,0.0,0,27338
2,56.0,98.0,36.225,127.0,105.0,73.0,15.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,3.0,0.0,0,27338
3,54.0,98.0,36.250,119.0,100.0,76.0,13.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,4.0,0.0,0,27338
4,52.0,98.0,36.275,119.0,87.0,67.0,12.0,33.017979,-0.441724,24.382587,...,194.00,57.0,0.0,0.492043,0.507957,-12.02,5.0,0.0,0,27338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429849,88.0,95.0,37.300,126.0,82.0,61.0,15.0,33.017979,0.800000,26.680000,...,81.96,76.7,0.0,0.000000,1.000000,-19.52,39.0,0.0,0,18000
1429850,88.0,98.0,37.400,118.0,75.0,57.0,20.0,33.017979,1.600000,26.760000,...,82.72,76.7,0.0,0.000000,1.000000,-19.52,40.0,0.0,0,18000
1429851,92.0,96.0,37.400,143.0,96.0,70.0,13.0,33.017979,2.400000,26.840000,...,83.48,76.7,0.0,0.000000,1.000000,-19.52,41.0,0.0,0,18000
1429852,95.0,96.0,37.600,129.0,84.0,61.0,15.5,33.017979,3.200000,26.920000,...,84.24,76.7,0.0,0.000000,1.000000,-19.52,42.0,0.0,0,18000


In [106]:
badcols = ["ICULOS", "SepsisLabel", "SepsisEver", "index"]
slim = df[[i for i in df.columns if i not in badcols]]
variations = slim.groupby("id").std().mean()
static_vars = variations[variations == 0].index.tolist()
puny_vars = variations[(0 < variations) & (variations < 0.05)].index.tolist()
moving_vars = [i for i in variations.index.tolist() if i not in puny_vars + static_vars]

In [107]:
data_groups = {
    "earlymean": slim[moving_vars + ["id"]].groupby("id").head(npts).groupby("id").mean().reset_index(),
    "earlystd": slim[moving_vars + ["id"]].groupby("id").head(npts).groupby("id").std().reset_index(),
    "labels": df.groupby("id")["SepsisEver"].tail(1),
    "skew": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").skew().reset_index(),
    "std": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").std().reset_index(),
    "mean": slim[moving_vars + ["id"]].groupby("id").tail(npts).groupby("id").mean().reset_index(),
    "static": slim[static_vars + ["id"]].groupby("id").tail(1).reset_index()
}

In [113]:
var_sets = {
    "late": ["std", "mean", "static", "skew"],
    "early": ["earlymean", "earlystd", "static"],
    "static": ["static"],
    "moving": ["mean", "std", "skew"],
    "tail": ["mean"],
}

var_sets["kitchen_sink"] = var_sets["late"] + var_sets["early"]

        
def assemble_data(var_set):
    chosen_sets = var_sets[var_set]

    dataset = data_groups[chosen_sets[0]].copy()
    dataset.columns = [
        (i + "_" + chosen_sets[0]).replace(f"id_{chosen_sets[0]}", "id")
        for i in dataset.columns
    ]
    
    for i in range(1, len(chosen_sets)):
        dataset = dataset.merge(
            data_groups[chosen_sets[i]], on="id", suffixes=("", "_" + chosen_sets[i])
        )
    
    del dataset["id"]
    for i in dataset.columns:
        if "index" in i:
            del dataset[i]
        
    return dataset

assemble_data("kitchen_sink")

,HR_std,O2Sat_std,Temp_std,SBP_std,MAP_std,DBP_std,Resp_std,EtCO2_std,BaseExcess_std,HCO3_std,...,Hgb_earlystd,PTT_earlystd,WBC_earlystd,Fibrinogen_earlystd,Platelets_earlystd,Age_static,Gender_static,Unit1_static,Unit2_static,HospAdmTime_static
0,3.055505,1.012423,0.124601,20.773932,9.890310,4.761244,2.006240,0.0,0.421637,0.125008,...,0.023784,0.202160,0.107026,0.0,2.378354,63.18,0.0,0.492043,0.507957,-0.02
1,3.852849,1.950071,0.193196,7.761908,6.290204,4.412419,3.162717,0.0,0.000000,0.079057,...,0.000000,2.204339,0.000000,0.0,0.000000,58.67,1.0,0.000000,1.000000,-72.06
2,16.593506,2.945807,0.179547,0.000000,9.104486,0.000000,2.044641,0.0,0.000000,0.237537,...,0.000000,0.000000,0.000000,0.0,0.000000,76.16,0.0,0.492043,0.507957,-52.14
3,4.655045,0.761942,0.376509,13.913323,7.333523,5.337498,3.093003,0.0,0.000000,0.000000,...,0.473984,6.824056,1.289315,0.0,18.961526,78.63,1.0,0.000000,1.000000,-134.72
4,2.828918,0.447989,0.145340,17.176103,14.045664,0.000000,3.717825,0.0,0.000000,0.000000,...,0.065405,0.000000,0.081756,0.0,2.943213,53.62,1.0,1.000000,0.000000,-0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36607,1.334375,0.579751,0.133229,2.536402,1.686548,1.817355,2.241651,0.0,0.000000,0.000000,...,0.287781,0.000000,0.104648,0.0,5.493997,41.00,1.0,0.000000,1.000000,0.00
36608,3.362832,1.219188,0.000000,9.119508,10.141529,8.508529,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,46.00,0.0,0.492043,0.507957,-4.51
36609,3.541814,1.932184,0.259821,7.004959,5.245104,4.141927,4.143268,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,47.00,1.0,0.492043,0.507957,-0.01
36610,4.195235,0.527046,0.073786,27.548745,14.704497,8.778762,1.779513,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,59.00,0.0,0.000000,1.000000,-12.64


In [73]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_validate

class BoostModel(object):
    
    def __init__(self, X, y, group="none", nparts=10):
        self.Xdata = X
        self.ydata = y
        self.nparts = nparts
        self.results = pd.DataFrame()
        self.dgroup = group
        
    def optimize(self):
        depths = [2, 3, 5, 10]
        min_leafs = [2, 8, 32, 128]
        
        for i in depths:
            for j in min_leafs:
                model = RFC(max_depth=i, min_samples_leaf=j, class_weight="balanced")
                cv_results = cross_validate(model, self.Xdata, self.ydata, cv=self.nparts, scoring=("f1", "accuracy"))
                self.results = self.results.append(
                    {
                        "depth": i,
                        "min_leafs": j,
                        "data_group": self.dgroup,
                        "test_acc": cv_results["test_accuracy"].mean(),
                        "test_acc_std": cv_results["test_accuracy"].std(),
                        "test_f1": cv_results["test_accuracy"].mean(),
                        "test_f1_std": cv_results["test_accuracy"].std(),
                    },
                    ignore_index=True
                )
                print(
                    "Random Forest with depth {} and min_leafs {} got accuracy = {} +/- {} with f1 = {} on {}.".format(
                        i, j, 
                        cv_results["test_accuracy"].mean(), cv_results["test_accuracy"].std(),
                        cv_results["test_f1"]
                        self.dgroup
                    )
                )


 
all_results = pd.DataFrame()
for i in var_sets:
    print("\nPreparing {}...".format(i))
    xdata = assemble_data(i)
    print("Data prepped. Running models.")
    BM = BoostModel(xdata, data_groups["labels"], group=i)
    BM.optimize()
    all_results = all_results.append(BM.results, ignore_index=True)
    all_results.to_csv("model_opt_results.csv", index=False)


Preparing late...
Data prepped. Running models.
Random Forest with depth 2 and min_leafs 2 got accuracy = 0.5839168999227393 +/- 0.2116174621477832 on late.
Random Forest with depth 2 and min_leafs 8 got accuracy = 0.5817868715530924 +/- 0.2051878102477961 on late.
Random Forest with depth 2 and min_leafs 32 got accuracy = 0.5763242413316086 +/- 0.194665042818923 on late.
Random Forest with depth 2 and min_leafs 128 got accuracy = 0.585802772101047 +/- 0.20175314575051811 on late.
Random Forest with depth 2 and min_leafs 256 got accuracy = 0.5903115947077339 +/- 0.2204703558668445 on late.
Random Forest with depth 3 and min_leafs 2 got accuracy = 0.6183317716626058 +/- 0.2064032958121587 on late.
Random Forest with depth 3 and min_leafs 8 got accuracy = 0.6166366639908667 +/- 0.18356403097792412 on late.
Random Forest with depth 3 and min_leafs 32 got accuracy = 0.6136072564953543 +/- 0.20620080918022354 on late.
Random Forest with depth 3 and min_leafs 128 got accuracy = 0.5989394090

In [115]:
late_data = assemble_data("late")
late_model = RFC(max_depth=10, min_samples_leaf=5, class_weight="balanced")
late_model.fit(late_data, data_groups["labels"])

early_data = assemble_data("early")
early_model = RFC(max_depth=10, min_samples_leaf=5, class_weight="balanced")
early_model.fit(early_data, data_groups["labels"])

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=5)

In [116]:
import numpy as np

def get_features(model, data):
    features = pd.DataFrame(
        data=np.asarray([data.columns.tolist(), model.feature_importances_.tolist()]).T, 
        columns=["feature", "importance"]
    )
    return features.sort_values(by="importance", ascending=False).reset_index(drop=True)

print("late:")
print(get_features(late_model, late_data).head(20))

print("\nearly:")
print(get_features(early_model, early_data).head(20))

late:
       feature            importance
0    Platelets  0.023320114880305807
1    Temp_skew   0.02145039735667789
2         Temp  0.020937590173274012
3          SBP   0.02085242318912929
4       HR_std  0.020743057655751875
5           HR  0.020602718989273558
6      MAP_std  0.020384448805610106
7      Calcium  0.020152801476564477
8     Temp_std  0.020141826136118413
9          Hct  0.019833759065901128
10         WBC  0.019770599361059536
11     Glucose   0.01972158767116206
12   Magnesium   0.01951566706671687
13        Resp  0.019000566190767808
14    DBP_skew  0.018939093392486974
15         MAP  0.018832892776411956
16   O2Sat_std  0.018200966230095674
17  O2Sat_skew    0.0176704313598073
18   Potassium  0.017471725799557032
19    MAP_skew   0.01738108992433526

early:
                 feature            importance
0    Potassium_earlymean  0.029773307779178665
1      Calcium_earlymean   0.02639284562018768
2         Temp_earlymean   0.02575718271635078
3           HR_earlym

In [127]:
xdata = assemble_data("late")
print("Data prepped. Running models.")
BM = BoostModel(xdata, data_groups["labels"], group="late")
BM.optimize()
all_results = all_results.append(BM.results, ignore_index=True)
all_results.to_csv("model_opt_results.csv", index=False)

Data prepped. Running models.
Random Forest with depth 2 and min_leafs 2 got accuracy = 0.5501025168085347 +/- 0.16610075239913497 on late.
Random Forest with depth 2 and min_leafs 8 got accuracy = 0.5787804900607776 +/- 0.17618248025871305 on late.
Random Forest with depth 2 and min_leafs 32 got accuracy = 0.5906648838607782 +/- 0.1841090159776819 on late.
Random Forest with depth 2 and min_leafs 128 got accuracy = 0.5855578550893882 +/- 0.18923345223568977 on late.
Random Forest with depth 2 and min_leafs 256 got accuracy = 0.5949539934936436 +/- 0.19787912369917937 on late.
Random Forest with depth 3 and min_leafs 2 got accuracy = 0.6392799671086934 +/- 0.1660488633067135 on late.
Random Forest with depth 3 and min_leafs 8 got accuracy = 0.622562566655692 +/- 0.14324658080683983 on late.
Random Forest with depth 3 and min_leafs 32 got accuracy = 0.6156278087882504 +/- 0.16565692507454569 on late.
Random Forest with depth 3 and min_leafs 128 got accuracy = 0.6074882098957064 +/- 0.17